In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]

In [2]:
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=3)
search.invoke("tell me about 2025 LA wildfire updates")

[{'url': 'https://apnews.com/live/los-angeles-wildfires-updates-1-13-25',
  'content': 'Los Angeles wildfires: Updates from Jan. 13, 2025 | AP News 7 of 10\xa0|\xa0Los Angeles Fire Department’s Mike Alvarez works on extinguishing hot spots in the aftermath of the Palisades Fire as a Malibu resident, top right, watches the sunset from atop his beachfront home along the Pacific Coast Highway in Malibu, Calif., Sunday, Jan. 12, 2025. 7 of 10\xa0|\xa0Los Angeles Fire Department’s Mike Alvarez works on extinguishing hot spots in the aftermath of the Palisades Fire as a Malibu resident, top right, watches the sunset from atop his beachfront home along the Pacific Coast Highway in Malibu, Calif., Sunday, Jan. 12, 2025. “We’re absolutely better prepared for this coming,” Los Angeles County Fire Chief Anthony Marrone said when asked what will be different this time with similarly strong winds forecast as last week which fueled the massive fires.'},
 {'url': 'https://recovery.lacounty.gov/2025/0

In [9]:
tools = [search]

In [10]:
tools

[TavilySearchResults(max_results=3)]

In [21]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [22]:
from langchain_core.messages import HumanMessage
response = agent_executor.invoke({"messages": [HumanMessage(content="tell me about 2025 LA wildfire updates")]})
response

{'messages': [HumanMessage(content='tell me about 2025 LA wildfire updates', id='61bac94b-f076-4dda-9af4-e44f21b17b1a'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GrUHojcZo20B2pvJhC2usbPN', 'function': {'arguments': '{"query":"2025 LA wildfire updates"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 90, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0a5af93e-68df-482e-aa29-2ca43625d825-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025 LA wildfire updates'}, 'id': 'call_GrUHojcZo20B2pvJhC2usbPN', 'type': 'tool_call'}], usage_metadata={'input

In [23]:
response["messages"]

[HumanMessage(content='tell me about 2025 LA wildfire updates', id='61bac94b-f076-4dda-9af4-e44f21b17b1a'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GrUHojcZo20B2pvJhC2usbPN', 'function': {'arguments': '{"query":"2025 LA wildfire updates"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 90, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0a5af93e-68df-482e-aa29-2ca43625d825-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025 LA wildfire updates'}, 'id': 'call_GrUHojcZo20B2pvJhC2usbPN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [15]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [16]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [17]:
config = {"configurable": {"thread_id": "001"}}

In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UEgG16mvYiSPrP2eCTKcIkYZ', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 91, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c91979be-03ad-4840-98df-fba9130d4d43-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_UEgG16mvYiSPrP2eCTKcIkYZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 24, 'total_tokens': 115})]}}
----
{'tools': {'messages': [ToolMe

In [26]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_m0tbNUOYWsEkJ32B9ZsATvCw', 'function': {'arguments': '{"query":"top stars of the winning team"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 91, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1d2de8a4-0276-41e9-9be3-b5166f83ffc4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'top stars of the winning team'}, 'id': 'call_m0tbNUOYWsEkJ32B9ZsATvCw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 24, 'total_tokens': 115})]}}
----
{'tools': {'messages': [

In [28]:
config = {"configurable": {"thread_id": "001"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You mentioned the soccer team "Manchester United" in our previous conversation. Would you like me to provide more information about them?', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 89, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b9fdab43-09da-4e58-9b4e-caff79772e3d-0', usage_metadata={'input_tokens': 89, 'output_tokens': 27, 'total_tokens': 116})]}}
----
